<font face=楷体 size=6><b>黑人抬棺人脸检测:</b>

<font face=楷体 size=5><b>背景:</b>
  
<font face=楷体 size=3>黑人抬棺这么火，怎么能不用paddlehub试一试呢？
<br>
<font face=楷体 size=3>临近期末，准备考试，还要准备考研，555，明明有好点子，但是没时间做，先出一个黑人抬棺的视频8
  
<font face=楷体 size=5><b>结果:</b>
  
<font face=楷体 size=3>在我的B站上: <a href=https://www.bilibili.com/video/BV1Sk4y1r7Zz>https://www.bilibili.com/video/BV1Sk4y1r7Zz</a>
  
<font face=楷体 size=5><b>思路和步骤:</b>
  
<font face=楷体 size=3>思路嘛，再简单不过，一帧一帧拆分，一帧一帧人脸检测
<font face=楷体 size=3>步骤嘛，人脸检测 + ffmpeg拆分合并
  
<font face=楷体 size=5><b>总结:</b><br>
<font face=楷体 size=3>paddlehub蛮好用的，改日有时间定要搞一番事业<br>
<font face=楷体 size=3>时间太少了，考研党伤不起啊啊啊

In [1]:
from IPython.display import HTML
HTML('<iframe style="width:98%;height: 450px;" src="//player.bilibili.com/player.html?bvid=BV1Sk4y1r7Zz" scrolling="no" border="0" frameborder="no" framespacing="0" allowfullscreen="true"> </iframe>')

In [2]:
# ---------------------------------------------------------------------------
# 为使用 `GPU` 设置环境变量 , (仍然报错, 已在github上提交issue——目前已解决)
# ---------------------------------------------------------------------------
%set_env CUDA_VISIBLE_DEVICES = 0

env: CUDA_VISIBLE_DEVICES=0


In [3]:
# ---------------------------------------------------------------------------
# 安装视频处理环境
# ---------------------------------------------------------------------------
!pip install moviepy -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install ffmpeg

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/


In [4]:
# ---------------------------------------------------------------------------
# 安装paddlehub环境和下载模型
# ---------------------------------------------------------------------------

try:
    import paddlehub as hub

except ImportError:
    !pip install paddlehub==1.6.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
    import paddlehub as hub

try:
    module = hub.Module(name="ultra_light_fast_generic_face_detector_1mb_640")
    # module = hub.Moudle(name="ultra_light_fast_generic_face_detector_1mb_320")
except FileNotFoundError:
    !hub install ultra_light_fast_generic_face_detector_1mb_640==1.1.2
    module = hub.Module(name="ultra_light_fast_generic_face_detector_1mb_640")
    # module = hub.Moudle(name="ultra_light_fast_generic_face_detector_1mb_320")

[2020-05-07 14:22:40,518] [    INFO] - Installing ultra_light_fast_generic_face_detector_1mb_640 module
[2020-05-07 14:22:40,527] [    INFO] - Module ultra_light_fast_generic_face_detector_1mb_640 already installed in /home/aistudio/.paddlehub/modules/ultra_light_fast_generic_face_detector_1mb_640


注:
Ultra-Light-Fast-Generic-Face-Detector-1MB提供了两种预训练模型，ultra_light_fast_generic_face_detector_1mb_320和ultra_light_fast_generic_face_detector_1mb_640。

- ultra_light_fast_generic_face_detector_1mb_320，在预测时会将图片输入缩放为320 * 240，预测速度更快。关于该模型更多介绍， 查看[PaddleHub官网介绍](https://www.paddlepaddle.org.cn/hubdetail?name=ultra_light_fast_generic_face_detector_1mb_320&en_category=ObjectDetection)
- ultra_light_fast_generic_face_detector_1mb_640，在预测时会将图片输入缩放为640 * 480，预测精度更高。关于该模型更多介绍， 查看[PaddleHub官网介绍](https://www.paddlepaddle.org.cn/hubdetail?name=ultra_light_fast_generic_face_detector_1mb_640&en_category=ObjectDetection)

用户根据需要，选择具体模型。利用PaddleHub使用该模型时，只需更改指定name，即可实现无缝切换。

In [5]:
# ---------------------------------------------------------------------------
# 查看黑人抬棺视频的基本信息
# ---------------------------------------------------------------------------
import os
import cv2
import json
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tqdm import tqdm

video = cv2.VideoCapture("video.flv")

fps = video.get(cv2.CAP_PROP_FPS)                # 视频帧率
frameCount = video.get(cv2.CAP_PROP_FRAME_COUNT) # 获得视频的总帧数
width = video.get(cv2.CAP_PROP_FRAME_WIDTH)      # 获得视频的宽度
height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)    # 获得视频的高度

print('视频的宽度:{}'.format(width))
print('视频的高度:{}'.format(height))
print('视频帧率:{}'.format(fps))
print('视频的总帧数:{}'.format(frameCount))

视频的宽度:852.0
视频的高度:480.0
视频帧率:30.06896551724138
视频的总帧数:1330.0


In [6]:
cv2.__version__

'4.1.1'

In [7]:
# ---------------------------------------------------------------------------
# 将视频数据变为帧数据, 并且保存
# ---------------------------------------------------------------------------
if not os.path.exists('frame'):
    os.mkdir('frame')

all_img = []
all_img_path_dict = {'image':[]}

success, frame = video.read()
i = 0
while success:
    
    all_img.append(frame)
    i += 1
    # if not i % 10:print(i)
    success, frame = video.read()

    path = os.path.join('frame', str(i)+'.jpg')
    all_img_path_dict['image'].append(path)
    cv2.imwrite(path, frame)

all_img_path_dict['image'].pop()
print('完毕')

完毕


In [8]:
# ---------------------------------------------------------------------------
# 预测并打印输出(或者找到已经保存的文件)
# ---------------------------------------------------------------------------

# 读取视频所保存的信息文件
info_path = 'info.json'

# hub版本更新后, 其检测精度浮点数太飘了, 这里赶时间, 就暂时不写了(准备期末考试ing...)
# if os.path.exists(info_path):

    # # 读取已经保存的`json`数据
    # with open(info_path, 'r') as f:
    #     json_dict = json.load(f)
    # results = json_dict['data']
if False:
    pass

else: # 若没有找到`json`数据

    # PaddleHub对于支持一键预测的module，可以调用module的相应预测API，完成预测功能。
    results = module.face_detection(data=all_img_path_dict,
                                    use_gpu=True,
                                    visualization=True)
    # save_json = {'data':results}
    # with open(info_path, 'w') as f:
    #     f.write(json.dumps(save_json))


[2020-05-07 14:22:56,227] [    INFO] - 0 pretrained paramaters loaded by PaddleHub


In [9]:
# ---------------------------------------------------------------------------
# 输出制作视频文件的备用文件
# ---------------------------------------------------------------------------

# 输出视频的size
size = (int(width), int(height))
size = (int(height), int(width))
# 创建写视频对象(不好用)
# videoWriter = cv2.VideoWriter("a.avi", cv2.VideoWriter_fourcc('M','J','P','G'), fps, size) 

for i, info in tqdm(enumerate(results)):
    
    num_info = info['data']

    if not len(num_info):
        # 如果该画面没有人, 则 `frame`变量赋值为原来的图片
        frame = all_img[i].copy()[:,:, ::-1]
    else:
        # frame = mpimg.imread(info['save_path'])   # 之前的save_path现在没了......
        frame = mpimg.imread(info['path'].replace('frame', 'face_detector_640_predict_output'))
    
    cv2.putText(frame, 'fps: {:.2f}'.format(fps), (20, 370), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0 ,255), 2)
    cv2.putText(frame, 'count: ' + str(len(num_info)), (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0 ,255), 2)
    cv2.putText(frame, 'frame: ' + str(i), (20, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0 ,255), 2)
    # cv2.putText(frame, 'time: {:.2f}s'.format(i / fps), (20,460), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,255), 2)
    
    plt.imsave('./img_out/{}.jpg'.format(i), frame)


1323it [01:09, 19.03it/s]


In [10]:
# ---------------------------------------------------------------------------
# 输出视频文件(没加配乐, 没有灵魂)(找了一堆python工具, 不如ffmpeg好用)
# ---------------------------------------------------------------------------
if os.path.exists('temp.mp4'):
    !rm -f temp.mp4
!ffmpeg -f image2 -i img_out/%d.jpg  -vcodec libx264 -r 60.0  temp.mp4

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

In [11]:
# ---------------------------------------------------------------------------
# 抽离源文件配乐
# ---------------------------------------------------------------------------
if os.path.exists('nb.mp3'):
    !rm -f nb.mp3
!ffmpeg -i video.flv -f mp3 nb.mp3

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

In [12]:
# ---------------------------------------------------------------------------
# 音乐视频合成(由于需要调整视频速度, 使音频和视频时间一样, 命令行不太好调整, 我将合成放在了本地端)
# ---------------------------------------------------------------------------

# # 去掉temp视频音轨
# !ffmpeg -i temp.mp4 -c:v copy -an temp_new.mp4 
# # 给视频加背景音乐
# !ffmpeg -i temp_new.mp4 -i nb.mp3 -t 52 -y last.mp4